In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import os
logdir = 'log'

def write_graph(name):
    file_writer = tf.summary.FileWriter(os.path.join(logdir, name))
    with tf.Session() as sess:
        file_writer.add_graph(sess.graph)
    file_writer.close()

In [3]:
n_classes = 10
num_layers = 2
state_size = 25
keep_prob = 0.5
vocab_size = 1000
embedding_size = 150
embedding_vectors = np.random.rand(vocab_size, embedding_size).astype(np.float32)

# Inputs and embeddings

In [4]:
x = tf.placeholder(tf.int32, [None, None], name='x')
seqlen = tf.placeholder(tf.int32, [None], name='seqlen')

W = tf.Variable(
        embedding_vectors,
        trainable=False,
        name="word_embeddings")
rnn_inputs = tf.nn.embedding_lookup(W, x)

write_graph("step_1")

# LSTM sequence labeling

## Defining the LSTM cell

In [5]:
def lstm_cell_with_dropout(state_size, keep_prob):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    cell = tf.contrib.rnn.DropoutWrapper(
        cell,
        output_keep_prob=keep_prob,
        state_keep_prob=keep_prob,
        variational_recurrent=True,
        dtype=tf.float32)
    return cell

## Creating a bidirectional LSTM layer

In [10]:
def blstm_layer_with_dropout(inputs, seqlen, state_size, keep_prob, scope):
    cell = lstm_cell_with_dropout(state_size, keep_prob)
    (output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(
        cell,
        cell,
        inputs,
        sequence_length=seqlen,
        dtype=tf.float32,
        scope=scope)
    return tf.concat([output_fw, output_bw], axis=-1, name="blstm_output")

In [7]:
for i in range(num_layers):
    with tf.name_scope("BLSTM-{}".format(i)) as scope:
        rnn_inputs = blstm_layer_with_dropout(
            rnn_inputs, seqlen, state_size, keep_prob, scope)
            
write_graph("step_2")

# Predictions

In [8]:
max_length = tf.shape(x)[1]

with tf.name_scope('logits'):
    logit_inputs = tf.reshape(rnn_inputs, [-1, 2 * state_size])
    logits = tf.layers.dense(logit_inputs, n_classes)
    logits = tf.reshape(logits, [-1, max_length, n_classes])

with tf.name_scope('predictions'):
    predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)

write_graph("step_3")

# Training

In [9]:
labels = tf.placeholder(tf.int32, [None, None], name='labels')

with tf.name_scope('loss'):
    seqlen_mask = tf.sequence_mask(
            seqlen, maxlen=max_length, name='sequence_mask')
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=labels, logits=logits, name="cross_entropy")
    loss = tf.boolean_mask(loss, seqlen_mask)
    loss = tf.reduce_mean(loss, name="mean_loss")

with tf.name_scope('training'):
    #constant
    train_step = tf.train.AdamOptimizer(3e-4).minimize(loss)
    
write_graph("step_4")

/srv/devel/cvanweelden/anaconda3/envs/polyglot/lib/python3.5/site-packages/tensorflow-1.6.0-py3.5-linux-x86_64.egg/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
